In [1]:
import spacy
from spacy.lang.tr import Turkish
from spacy.util import minibatch, compounding
import pandas as pd
import numpy as np
import string

In [2]:
#read tremo dataset
tremopath = '/Users/pinarayaz/Jupyter/NLP/data/tremo.csv'
tremo_df = pd.read_csv(tremopath)
tremo_df.head()

,Entry,ValidatedEmotion
0,her yeni gün bir mutluluk,Happy
1,gece kimsenin olmadığı sokaklardan geçerken ço...,Fear
2,gerçekleşemeyen hayaller,Sadness
3,arkadaş kaybetmek beni üzüyor,Sadness
4,insanların çıkarcı olmalarından tiksiniyorum,Disgust


In [3]:
nlp = Turkish()
sample_review = tremo_df.Entry[17]
sample_review

'Geçerim dediğim dersten kalma durumuna gelmem beni çok şaşırtıyor.'

In [4]:
parsed_review = nlp(sample_review)
parsed_review

Geçerim dediğim dersten kalma durumuna gelmem beni çok şaşırtıyor.

In [5]:
tokenized_text = pd.DataFrame()

for i, token in enumerate(parsed_review):
    tokenized_text.loc[i, 'text'] = token.text
    tokenized_text.loc[i, 'lemma'] = token.lemma_,
    tokenized_text.loc[i, 'pos'] = token.pos_
    tokenized_text.loc[i, 'tag'] = token.tag_
    tokenized_text.loc[i, 'dep'] = token.dep_
    tokenized_text.loc[i, 'shape'] = token.shape_
    tokenized_text.loc[i, 'is_alpha'] = token.is_alpha
    tokenized_text.loc[i, 'is_stop'] = token.is_stop
    tokenized_text.loc[i, 'is_punctuation'] = token.is_punct

tokenized_text[:10]

,text,lemma,pos,tag,dep,shape,is_alpha,is_stop,is_punctuation
0,Geçerim,Geçerim,,,,Xxxxx,True,False,False
1,dediğim,"(de,)",,,,xxxx,True,False,False
2,dersten,"(ders,)",,,,xxxx,True,False,False
3,kalma,"(kal,)",,,,xxxx,True,False,False
4,durumuna,"(durum,)",,,,xxxx,True,False,False
5,gelmem,"(gel,)",,,,xxxx,True,False,False
6,beni,"(ben,)",,,,xxxx,True,True,False
7,çok,"(çok,)",,,,xxx,True,True,False
8,şaşırtıyor,"(şaşır,)",,,,xxxx,True,False,False
9,.,"(.,)",,,,.,False,False,True


In [6]:
tremo_df['tuples'] = tremo_df.apply(lambda row: (row['Entry'],row['ValidatedEmotion']), axis=1)
train = tremo_df['tuples'].tolist()
train[:10]

[('her yeni gün bir mutluluk', 'Happy'),
 ('gece kimsenin olmadığı sokaklardan geçerken çok korkarım', 'Fear'),
 ('gerçekleşemeyen hayaller', 'Sadness'),
 ('arkadaş kaybetmek beni üzüyor', 'Sadness'),
 ('insanların çıkarcı olmalarından tiksiniyorum', 'Disgust'),
 ('hiç beklemediğim anda sürprizle karşılaşmak beni şaşırtır', 'Surprise'),
 ('Ailemle tatile çıktığımda çok sevindim.', 'Happy'),
 ('Rüyamda babamın öldüğünü gördüğümde çok korkmuştum.', 'Fear'),
 ('Daha taksiti bitmemiş bilgisayarımı ben uyurken otobüsten çalınmasına çok öfkelendim.',
  'Anger'),
 ('Annemin üzüldüğünü gördüğümde üzülüyorum.', 'Sadness')]

In [7]:
nlp = spacy.blank("tr")  # create blank Language class

if "textcat" not in nlp.pipe_names:
    textcat = nlp.create_pipe("textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})
    nlp.add_pipe(textcat, last=True)
else:
    textcat = nlp.get_pipe("textcat")

In [8]:
textcat.add_label("Happy")
textcat.add_label("Fear")
textcat.add_label("Sadness")
textcat.add_label("Disgust")
textcat.add_label("Surprise")
textcat.add_label("Anger")

1

In [9]:
def load_data(limit=0, split=0.8):
    train_data = train
    np.random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{"Happy": y == "Happy", "Fear": y == "Fear", "Sadness": y == "Sadness",
             "Disgust": y == "Disgust", "Suprise": y == "Suprise", "Anger": y == "Anger"} for y in labels]
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])

In [10]:
# load the dataset
print("Loading TREMO data...")
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(limit=27350)
print("Using {} examples ({} training, {} evaluation)".format(27350, len(train_texts), len(dev_texts)))
train_data = list(zip(train_texts, [{'cats': cats} for cats in train_cats]))

Loading TREMO data...
Using 27350 examples (21880 training, 5470 evaluation)


In [11]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score, 'textcat_a': accuracy}

In [12]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()
    print("Training the model...")
    print('{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F', 'Accuracy'))
    for i in range(10):
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
        with textcat.model.use_params(optimizer.averages):
            # evaluate on the dev data split off in load_data()
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}\t{4:.3f}'
              .format(losses['textcat'], scores['textcat_p'],
                      scores['textcat_r'], scores['textcat_f'], scores['textcat_a']))

Training the model...
LOSS 	  P  	  R  	  F  	Accuracy
17.012	0.900	0.767	0.829	0.946
8.321	0.880	0.797	0.837	0.947
6.111	0.875	0.812	0.842	0.949
4.554	0.867	0.822	0.844	0.949
3.671	0.860	0.820	0.840	0.947
3.040	0.854	0.826	0.840	0.947
2.668	0.854	0.826	0.840	0.947
2.419	0.848	0.825	0.836	0.945
1.970	0.847	0.826	0.836	0.945
1.790	0.844	0.828	0.836	0.945


In [15]:
# test the trained model
test_text = "Tahlil sonuçlarının kötü çıkmasından korkuyorum."
doc = nlp(test_text)
print(test_text, doc.cats)

Tahlil sonuçlarının kötü çıkmasından korkuyorum. {'Happy': 6.583996015052662e-09, 'Fear': 0.9999990463256836, 'Sadness': 3.6789039370432874e-08, 'Disgust': 7.194476552285778e-07, 'Surprise': 1.0683135798217336e-07, 'Anger': 6.638094163236019e-08}


In [16]:
#save model
output_dir = '/Users/pinarayaz/Jupyter/NLP/spacy_models/tremo_spacy'
if output_dir is not None:
    with nlp.use_params(optimizer.averages):
        nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /Users/pinarayaz/Jupyter/NLP/spacy_models/tremo_spacy


In [18]:
# test the saved model
test_text = "Hayallerim yıkıldı."
nlp_test = spacy.load(output_dir)
doc2 = nlp_test(test_text)
print(test_text, doc2.cats)

score = 0
for cat in doc2.cats:
    if(doc2.cats[cat] > score):
        emotion = cat
        score = doc2.cats[cat]
print("Emotion:", emotion)

Hayallerim yıkıldı. {'Happy': 0.12280184030532837, 'Fear': 0.0021183679345995188, 'Sadness': 0.6602464318275452, 'Disgust': 0.006913020741194487, 'Surprise': 0.17555099725723267, 'Anger': 0.03236926719546318}
Emotion: Sadness
